In [2]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  
from keras.preprocessing import sequence
from keras.regularizers import l2
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [5]:
traindata = pd.read_csv('kddtrain.csv', header=None)
traindata.columns= ["threat", "duration","protocol_type","service","flag","src_bytes","dst_bytes","land"
                         ,"wrong_fragment","urgent","hot","num_failed_logins","logged_in", 
                          "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations"
                         ,"num_shells", "num_access_files","num_outbound_cmds","is_host_login","is_guest_login"
                         ,"count", "srv_count","serror_rate", "srv_serror_rate","rerror_rate","srv_rerror_rate"
                         ,"same_srv_rate","diff_srv_rate", "srv_diff_host_rate","dst_host_count","dst_host_srv_count"
                         ,"dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate"
                         ,"dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate", "dst_host_srv_rerror_rate"]
testdata = pd.read_csv('kddtest.csv', header=None)
testdata.columns = traindata.columns
traindata = traindata.drop(columns = ["num_outbound_cmds","is_host_login"])
testdata = testdata.drop(columns = ["num_outbound_cmds","is_host_login"])
traindata.head()

,threat,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,0,1,1,1,181,5450,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,0,1,1,1,239,486,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,0,1,1,1,235,1337,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,0,1,1,1,219,1337,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,0,1,1,1,217,2032,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [6]:
testdata.head()

,threat,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,0,2,24,1,105,146,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
1,0,0,2,24,1,105,146,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
2,0,0,2,24,1,105,146,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
3,1,0,2,24,1,105,146,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
4,1,0,2,24,1,105,146,0,0,0,...,255,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0


In [7]:
X = traindata.iloc[:,1:40]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:40]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)


X_train = np.array(trainX)
X_test = np.array(testT)


batch_size = 64


In [3]:
l2_lambda = 0.0001
model = Sequential()
model.add(Dense(512,input_dim=39, kernel_regularizer=l2(l2_lambda), activation='relu'))  
model.add(Dropout(0.01))
model.add(BatchNormalization())
model.add(Dense(256, kernel_regularizer=l2(l2_lambda),activation='relu'))  
model.add(Dropout(0.01))
model.add(BatchNormalization())
model.add(Dense(128, kernel_regularizer=l2(l2_lambda), activation='relu'))  
model.add(Dropout(0.01))
model.add(BatchNormalization())
model.add(Dense(1, kernel_regularizer=l2(l2_lambda)))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="C:/Users/Shigatau/Documents/Диплом/Intrusion-Detection-Systems-master/dnn/kddresults/dnn3layer/check.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('C:/Users/Shigatau/Documents/Диплом/Intrusion-Detection-Systems-master/dnn/kddresults/dnn3layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train,  batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("C:/Users/Shigatau/Documents/Диплом/dnn/kddresults/dnn3layer/dnn3layer_model.hdf5")

Epoch 1/100
7720/7720 [==============================] - 46s 5ms/step - loss: 0.1769 - accuracy: 0.9922

Epoch 00001: loss improved from inf to 0.08412, saving model to C:/Users/Shigatau/Documents/Диплом/Intrusion-Detection-Systems-master/dnn/kddresults/dnn3layer\check.hdf5
Epoch 2/100
7720/7720 [==============================] - 36s 5ms/step - loss: 0.0185 - accuracy: 0.9966

Epoch 00002: loss improved from 0.08412 to 0.01695, saving model to C:/Users/Shigatau/Documents/Диплом/Intrusion-Detection-Systems-master/dnn/kddresults/dnn3layer\check.hdf5
Epoch 3/100
7720/7720 [==============================] - 35s 5ms/step - loss: 0.0156 - accuracy: 0.9968

Epoch 00003: loss improved from 0.01695 to 0.01474, saving model to C:/Users/Shigatau/Documents/Диплом/Intrusion-Detection-Systems-master/dnn/kddresults/dnn3layer\check.hdf5
Epoch 4/100
7720/7720 [==============================] - 36s 5ms/step - loss: 0.0144 - accuracy: 0.9969

Epoch 00004: loss improved from 0.01474 to 0.01315, saving mod

7720/7720 [==============================] - 42s 5ms/step - loss: 0.0124 - accuracy: 0.9972

Epoch 00088: loss did not improve from 0.00999
Epoch 89/100
7720/7720 [==============================] - 47s 6ms/step - loss: 0.0108 - accuracy: 0.9975

Epoch 00089: loss did not improve from 0.00999
Epoch 90/100
7720/7720 [==============================] - 40s 5ms/step - loss: 0.0102 - accuracy: 0.9976

Epoch 00090: loss did not improve from 0.00999
Epoch 91/100
7720/7720 [==============================] - 42s 5ms/step - loss: 0.0102 - accuracy: 0.9976

Epoch 00091: loss did not improve from 0.00999
Epoch 92/100
7720/7720 [==============================] - 46s 6ms/step - loss: 0.0103 - accuracy: 0.9978

Epoch 00092: loss did not improve from 0.00999
Epoch 93/100
7720/7720 [==============================] - 45s 6ms/step - loss: 0.0110 - accuracy: 0.9975

Epoch 00093: loss did not improve from 0.00999
Epoch 94/100
7720/7720 [==============================] - 44s 6ms/step - loss: 0.0106 - accurac

In [9]:
score = []
name = []
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import os
for file in os.listdir("C:/Users/Shigatau/Documents/Диплом/dnn/kddresults/dnn3layer/"):
    model.load_weights("C:/Users/Shigatau/Documents/Диплом/dnn/kddresults/dnn3layer/"+file)
    y_train1 = y_test
    y_pred = model.predict_classes(X_test)
    print(y_pred[0:3])
    print(y_train1[0:3])
    accuracy = accuracy_score(y_train1, y_pred)
    recall = recall_score(y_train1, y_pred , average="binary")
    precision = precision_score(y_train1, y_pred , average="binary")
    f1 = f1_score(y_train1, y_pred, average="binary")
    print("----------------------------------------------")
    print("accuracy")
    print("%.3f" %accuracy)
    print("recall")
    print("%.3f" %recall)
    print("precision")
    print("%.3f" %precision)
    print("f1score")
    print("%.3f" %f1)
    score.append(accuracy)
    name.append(file)

[[0]
 [0]
 [0]]
[0 0 0]
----------------------------------------------
accuracy
0.926
recall
0.909
precision
0.998
f1score
0.952


In [29]:
model.load_weights("C:/Users/Никита/Documents/Диплом/Intrusion-Detection-Systems-master/dnn/kddresults/dnn3layer/"+name[score.index(max(score))])
pred = model.predict_classes(X_test)
proba = model.predict_proba(X_test)
np.savetxt("dnnres/dnn3predicted.txt", pred)
np.savetxt("dnnres/dnn3probability.txt", proba)

accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred , average="binary")
precision = precision_score(y_test, pred , average="binary")
f1 = f1_score(y_test, pred, average="binary")

FileNotFoundError: [Errno 2] No such file or directory: 'dnnres/dnn3predicted.txt'